In [204]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import random

import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import re

In [2]:
def random_delay():
    time.sleep(random.uniform(2, 5))

### 1. Procèdure :

1. Aller sur la page de résultats avec selenium
2. Récupérer les URL pour aller de page en page
3. Faire une fonction pour aller récupérer les URL de chaque annonce sur chaque page
4. Couper les URL en relatif puis les reconcatener pour les passer dans l'API
5. Itérer pour requêter l'API avec chaque URL récupérée sur chaque page de résultats

1.1 Je teste sur la première page

In [94]:
## 1. je lance selenium pour aller sur la page de résultats ########################################################################
service = Service(executable_path = 'chromedriver.exe')                         # je dis où est mon driver                       
driver = webdriver.Chrome(service=service)                                      # je définis mon driver
driver.get("https://www.welcometothejungle.com/fr")                             # et je l'envoie sur la page d'accueil

wait = WebDriverWait(driver, 8)

input_element = driver.find_element(By.ID,"search-query-field")                 # je sélectionne la barre de recherche
input_element.clear()                                                           # je la vide au cas où elle est pré-remplie                                             
input_element.send_keys("Data analyst" + Keys.ENTER)                            # j'écris dans la barre de recherche

random_delay()                                                     # je mets un petit delay aléatoire 
                                                                   # pour laisser le temps à la page de charger

## 2. je vais chercher les url des annonces ########################################################################################
## je print un peu tout pour vérifier

links = (driver.find_elements(By.PARTIAL_LINK_TEXT, "Data")        # avec Selenium, je cherche les liens avec data dedans quelque soit la casse 
        or driver.find_elements(By.PARTIAL_LINK_TEXT, "DATA")
        or  driver.find_elements(By.PARTIAL_LINK_TEXT, "data")) 

urls = [link.get_attribute("href") for link in links]              # j'extraits le contenu de href pour le mettre dans la liste urls
      
print(f"J'ai {len(urls)} annonces")
print(urls)

## 3. je vais chercher les url de pagination ######################################################################################
## je print un peu tout pour vérifier

page_source = driver.page_source                                   # je vais chercher la page avec mon driver
soup = BeautifulSoup(page_source, "html.parser")                   #  je passe le code HTML à BeautifulSoup pour l'analyser

soup_pages = soup.find('nav', {"aria-label" : "Pagination"})      # Puis je vais chercher la section nav :

if soup_pages is not None:                                        # Je mets un if si la page n'est pas chargée et que nav est vide
    pli = soup_pages.find_all('li')                               # Puis je vais chercher tous les li contenus dans cette nav
    nb_pages = pli[-2].text                                       # Puis je vais chercher le texte de l'avant dernier li 
    print(f"\nJ'ai {nb_pages} pages")                               # (car le dernier c'est la flèche) pour avoir le nombre de pages                                         

else:                                                             # si la page n'a pas eu le temps de charger, j'affiche un message
    print("Aucun élément HTML correspondant au sélecteur spécifié n'a été trouvé.")


result_url = driver.current_url                                   # je récupère l'URL sur laquelle je suis
print("\nThe current url is: "+str(result_url))
base_url = result_url.replace("1","")

driver.quit()                                                     # je ferme le navigateur une fois tout fini


J'ai 28 annonces
['https://www.welcometothejungle.com/fr/companies/hellofresh/jobs/growth-data-analyst-intern-m-f-x_paris?q=10a8bef6c7bb9937e6f63ed5fb796130&o=9d259ad0-ec79-42d2-9626-bc788b09c4a2&p=true', 'https://www.welcometothejungle.com/fr/companies/stago/jobs/data-analyst-data-scientist-h-f_gennevilliers_STAGO_6KoyNwq?q=c33dfd12ebc20ecbecb329c1a7f8700d&o=f1553bbc-3576-42be-8b3f-8e9e16fd4297', 'https://www.welcometothejungle.com/fr/companies/the-product-crew/jobs/data-analyst_paris?q=c33dfd12ebc20ecbecb329c1a7f8700d&o=c2a8a9c2-28b3-4def-96d9-cd76aa12b530', 'https://www.welcometothejungle.com/fr/companies/numberly-1000mercis/jobs/stage-data-analyst_paris-9e-arrondissement?q=c33dfd12ebc20ecbecb329c1a7f8700d&o=2dba48ad-e509-441a-b125-331a613d1e0f', 'https://www.welcometothejungle.com/fr/companies/francetvpub/jobs/data-analyst-yield-numerique_boulogne-billancourt?q=c33dfd12ebc20ecbecb329c1a7f8700d&o=5fe79064-37fa-4c38-8dce-469ee0351222', 'https://www.welcometothejungle.com/fr/companies

Maintenant que j'ai le nombre de pages, je récupère les URL de toutes les pages de résultat

In [98]:
base_url = "https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type%5D%5B%5D=full_time&refinementList%5Bcontract_type%5D%5B%5D=temporary&page="
page_number = 1
urls_to_scrape = []

# je fais une boucle qui itère le nom de la page autant de fois que nécessaire (on aurait pu faire un while si on ne connaissait pas le nombre de page)
for i in range(1,int(nb_pages)):
  url = f"{base_url}{i}"
  urls_to_scrape.append(url)

print(urls_to_scrape)


['https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type%5D%5B%5D=full_time&refinementList%5Bcontract_type%5D%5B%5D=temporary&page=1', 'https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type%5D%5B%5D=full_time&refinementList%5Bcontract_type%5D%5B%5D=temporary&page=2', 'https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type%5D%5B%5D=full_time&refinementList%5Bcontract_type%5D%5B%5D=temporary&page=3', 'https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type%5D%5B%5D=full_time&refinementList%5Bcontract_type%5D%5B%5D=temporary&page=4', 'https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffices.country_code%

### 2. Après le test, je lance pour de bon cette 1e étape

Je commence par lancer le code pour récupérer les pages de résultats

In [33]:
############## SCRIPT PRINCIPAL COMPLET 1 ####################################
##############################################################################

## 1. je lance selenium pour aller sur la page de résultats ########################################################################
service = Service(executable_path = 'chromedriver.exe')                         # je dis où est mon driver                       
driver = webdriver.Chrome(service=service)                                      # je définis mon driver
driver.get("https://www.welcometothejungle.com/fr")                                                         # et je l'envoie sur la page d'accueil

wait = WebDriverWait(driver, 8)                                                 # je laisse le temps à la page de se charger

input_element = driver.find_element(By.ID,"search-query-field")                 # je sélectionne la barre de recherche
input_element.clear()                                                           # je la vide au cas où elle est pré-remplie                                             
input_element.send_keys("Data analyst" + Keys.ENTER)                            # j'écris dans la barre de recherche

random_delay()                                                                  # je laisse le temps à la page de charger

## 2. je vais chercher les url de pagination ######################################################################################
page_source = driver.page_source                                   # je vais chercher la page avec mon driver
soup = BeautifulSoup(page_source, "html.parser")                   #  je passe le code HTML à BeautifulSoup pour l'analyser

soup_pages = soup.find('nav', {"aria-label" : "Pagination"})      # Puis je vais chercher la section nav :

if soup_pages is not None:                                        # Je mets un if si la page n'est pas chargée et que nav est vide
    pli = soup_pages.find_all('li')                               # Puis je vais chercher tous les li contenus dans cette nav
    nb_pages = pli[-2].text                                       # Puis je vais chercher le texte de l'avant dernier li 
    print(f"\nJ'ai {nb_pages} pages")                               # (car le dernier c'est la flèche) pour avoir le nombre de pages

else:                                                             # si la page n'a pas eu le temps de charger, j'affiche un message
    print("Aucun élément HTML correspondant au sélecteur spécifié n'a été trouvé.")

result_url = driver.current_url                                   # je récupère l'URL sur laquelle je suis
base_url = result_url.replace("1","")

base_url = "https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type%5D%5B%5D=full_time&refinementList%5Bcontract_type%5D%5B%5D=temporary&page="
page_number = 1
urls_to_scrape = []

# 3. je fais une boucle qui itère le nom de la page autant de fois que nécessaire
for i in range(1,int(nb_pages)):
    url = f"{base_url}{i}"
    urls_to_scrape.append(url)
print(urls_to_scrape)

random_delay()     

jobs_list = []

for each_page in urls_to_scrape:
    try:
    # je charge la page actuelle dans le navigateur Web
      driver.get(each_page)
      random_delay() 

     # j'attends que la page soit entièrement chargée
      wait = WebDriverWait(driver, 20)
      wait.until(EC.visibility_of_element_located((By.PARTIAL_LINK_TEXT, "Data")))

      links = (driver.find_elements(By.PARTIAL_LINK_TEXT, "Data")        # avec Selenium, je cherche les liens avec data dedans quelque soit la casse 
        or driver.find_elements(By.PARTIAL_LINK_TEXT, "DATA")
        or  driver.find_elements(By.PARTIAL_LINK_TEXT, "data"))

      urls = [link.get_attribute("href") for link in links]              # j'extraits le contenu de href pour le mettre dans la liste urls
      print(urls)
    
      jobs_list.extend(urls)
    except:
        print(f"An error occured on page {driver.current_url}")  

print(f"J'ai {len(jobs_list)} annonces au total")
print(jobs_list)

driver.quit()                                                     # je ferme le navigateur une fois tout fini



J'ai 34 pages
['https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type%5D%5B%5D=full_time&refinementList%5Bcontract_type%5D%5B%5D=temporary&page=1', 'https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type%5D%5B%5D=full_time&refinementList%5Bcontract_type%5D%5B%5D=temporary&page=2', 'https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type%5D%5B%5D=full_time&refinementList%5Bcontract_type%5D%5B%5D=temporary&page=3', 'https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type%5D%5B%5D=full_time&refinementList%5Bcontract_type%5D%5B%5D=temporary&page=4', 'https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffice

Maintenant je forme mes URL comme il faut pour les passer par l'API

In [36]:
relative_urls = [sub.replace("https://www.welcometothejungle.com/fr/companies", "") for sub in jobs_list]
print(relative_urls)
print(len(relative_urls))

['/davidson/jobs/data-manager-h-f_DC_5KG03ZX?q=0c297f903b1befdac5f0bebf41aa2676&o=364981ca-b4b0-49f2-9cdc-9c10975b81f4', '/groupe-mentor/jobs/data-analyst-h-f_dommartemont_GM_rXWZkxG?q=0c297f903b1befdac5f0bebf41aa2676&o=110564e2-3db2-42c9-83e7-5510c4c3d8e4', '/groupe-mentor/jobs/data-analyst-h-f_dommartemont_GM_kLL8kRZ?q=0c297f903b1befdac5f0bebf41aa2676&o=e8b049a2-9682-4604-a09a-401f061038f0', '/papernest/jobs/data-analyst-stage-paris_paris?q=0c297f903b1befdac5f0bebf41aa2676&o=78f97400-3f69-4fb5-be5f-319c6aae92ea', '/hublo/jobs/senior-data-analyst-f-h-n_paris_HUBLO_bR85090?q=0c297f903b1befdac5f0bebf41aa2676&o=0b5d40e8-64f9-4643-969b-752d0fd2582c', '/adone/jobs/consultant-senior-data-bi-h-f_paris_AC_bNz65g0?q=0c297f903b1befdac5f0bebf41aa2676&o=823e76d1-9e76-4b00-9bfc-c499f1c7c0ee', '/hsbc/jobs/data-et-analytics-h-f_courbevoie?q=0c297f903b1befdac5f0bebf41aa2676&o=eff1ce84-17be-4859-9741-211a40dea708', '/blablacar/jobs/confirmed-data-analyst-blablacar-daily_paris?q=0c297f903b1befdac5f0beb

Je teste manuellement sur une url

In [53]:
# je requête une URL
link = 'https://api.welcometothejungle.com/api/v1/organizations/davidson/jobs/data-manager-h-f_DC_5KG03ZX?q=0c297f903b1befdac5f0bebf41aa2676&o=364981ca-b4b0-49f2-9cdc-9c10975b81f4'
r = requests.get(link)
print(r)

data_json = json.loads(r.text)
print(json.dumps(data_json, indent=2))

<Response [200]>
{
  "job": {
    "salary_max": null,
    "remote": "punctual",
    "application_fields": [
      {
        "id": "0e7960ee-b173-4aef-b52d-5c160fe82d22",
        "mode": "optional",
        "name": "cover_letter"
      },
      {
        "id": "6789a3be-baf8-40d4-b963-fe1235e30bab",
        "mode": "mandatory",
        "name": "resume"
      },
      {
        "id": "d464e154-98ad-4d09-87ab-91411aea9fbf",
        "mode": "disabled",
        "name": "portfolio"
      },
      {
        "id": "7d17fad8-6148-4a6d-ab97-ce41834fe5c6",
        "mode": "optional",
        "name": "picture"
      },
      {
        "id": "89798d58-34c8-4116-a2f2-bdbe33a86402",
        "mode": "optional",
        "name": "media_website"
      },
      {
        "id": "e71d4049-ba42-433c-a899-ff91792f4ffd",
        "mode": "optional",
        "name": "media_linkedin"
      },
      {
        "id": "5a96b1bc-1841-4aa0-9a59-12e244019fb7",
        "mode": "optional",
        "name": "media_twitter"


Je note ce que je veux garder

In [50]:
# j'en fais un dataframe
df_raw = pd.json_normalize(data_json)
df_raw

,job.remote,job.urls,job.language,job.slug,job.videos,job.updated_at,job.published_at,job.offices,job.experience_level,job.is_default,...,job.start_date,job.company_description,job.profile,job.tools,job.skills,job.recruitment_process,job.apply_url,job.wttj_reference,job.questions,job.contract_duration_min
0,punctual,"[{'kind': 'canonical', 'href': 'https://www.we...",fr,data-manager-h-f_DC_5KG03ZX,"[{'name': 'Rencontrez Jules, Lead culture tech...",2024-06-10T09:47:07Z,2024-06-10T09:47:07Z,"[{'address': '40 Rue Fanfan-la-Tulipe, 92100 B...",None,False,...,None,None,"<ul><li><p>De formation BAC+5, tu as déjà une ...","[{'name': 'Nosql', 'reference': '463c82d3-9c2d...",[{'name': {'cs': 'Umělá inteligence a strojové...,None,https://www.davidson.fr/nos-offres/data-manage...,364981ca-b4b0-49f2-9cdc-9c10975b81f4,[],None


In [54]:
# je regarde toutes les colonnes pour voir lesquelles je drop
df_raw.columns.values

array(['job.remote', 'job.urls', 'job.language', 'job.slug', 'job.videos',
       'job.updated_at', 'job.published_at', 'job.offices',
       'job.experience_level', 'job.is_default', 'job.reference',
       'job.featured_page.type', 'job.featured_page.slug',
       'job.cta_content.links', 'job.cta_content.contents', 'job.name',
       'job.salary_min', 'job.profile_type', 'job.contract_duration_max',
       'job.status', 'job.education_level', 'job.salary_period',
       'job.organization.name', 'job.organization.description',
       'job.organization.reference', 'job.organization.labels',
       'job.organization.logo.url', 'job.organization.logo.thumb.url',
       'job.organization.sectors',
       'job.organization.cover_image.small.url',
       'job.organization.cover_image.url',
       'job.organization.cover_image.medium.url',
       'job.organization.cover_image.large.url',
       'job.organization.cover_image.social.url',
       'job.organization.website_organization.slug',
 

In [60]:
df_job_temp = df_raw.drop(columns=["job.urls", 
                                   "job.slug", 
                                   "job.videos", 
                                   "job.updated_at",
                                   "job.offices", 
                                   "job.is_default",
                                   'job.organization.labels',
                                   'job.organization.logo.url', 
                                   'job.organization.logo.thumb.url',
                                   'job.organization.sectors',
                                   'job.organization.cover_image.small.url',
                                   'job.organization.cover_image.url',
                                   'job.organization.cover_image.medium.url',
                                   'job.organization.cover_image.large.url',
                                   'job.organization.cover_image.social.url',
                                   'job.organization.website_organization.slug',
                                   'job.organization.website_organization.i18n_descriptions.fr',
                                   'job.organization.gdpr_setting.application_message',
                                   'job.organization.gdpr_setting.consent_duration',
                                   'job.organization.gdpr_setting.privacy_policy_url',
                                   'job.organization.headquarter.address',
                                   'job.organization.headquarter.local_address',
                                   'job.organization.headquarter.city',
                                   'job.organization.headquarter.latitude',
                                   'job.organization.headquarter.longitude',
                                   'job.organization.headquarter.district',
                                   'job.organization.headquarter.country_code',
                                   'job.organization.headquarter.zip_code',
                                   'job.organization.headquarter.local_city',
                                   'job.organization.headquarter.local_district',
                                   'job.organization.media_facebook',
                                   'job.organization.media_instagram',
                                   'job.organization.media_linkedin',
                                   'job.organization.media_pinterest',
                                   'job.organization.media_twitter',
                                   'job.organization.media_youtube',
                                   'job.organization.playlist_id',
                                   'job.organization.video_playlist_provider',
                                   'job.organization.automatic_email',
                                   'job.archived_at',
                                   'job.profession.name.cs', 'job.profession.name.en',
                                   'job.profession.name.es', 'job.profession.name.sk',
                                   'job.profession.category.cs',
                                   'job.profession.category.en', 'job.profession.category.es',
                                   'job.profession.category.fr', 'job.profession.category.sk',
                                   'job.profession.category_name.cs',
                                   'job.profession.category_name.en',
                                   'job.profession.category_name.es',
                                   'job.profession.category_name.sk',
                                   'job.profession.sub_category_name.cs',
                                   'job.profession.sub_category_name.en',
                                   'job.profession.sub_category_name.es',
                                   'job.profession.sub_category_name.sk',
                                   'job.profession.sub_category_reference',
                                   'job.questions',
                                   "job.featured_page.type",
                                   'job.featured_page.slug',
                                   'job.cta_content.links',
                                   'job.cta_content.contents',
                                   'job.application_fields',
                                   ]
                                   )

In [62]:
df_job_temp

,job.remote,job.language,job.published_at,job.experience_level,job.reference,job.name,job.salary_min,job.profile_type,job.contract_duration_max,job.status,...,job.salary_currency,job.start_date,job.company_description,job.profile,job.tools,job.skills,job.recruitment_process,job.apply_url,job.wttj_reference,job.contract_duration_min
0,punctual,fr,2024-06-10T09:47:07Z,None,DC_5KG03ZX,Data Manager (H/F),None,standard,None,published,...,None,None,None,"<ul><li><p>De formation BAC+5, tu as déjà une ...","[{'name': 'Nosql', 'reference': '463c82d3-9c2d...",[{'name': {'cs': 'Umělá inteligence a strojové...,None,https://www.davidson.fr/nos-offres/data-manage...,364981ca-b4b0-49f2-9cdc-9c10975b81f4,None


In [64]:
df_job_temp.T

,0
job.remote,punctual
job.language,fr
job.published_at,2024-06-10T09:47:07Z
job.experience_level,None
job.reference,DC_5KG03ZX
job.name,Data Manager (H/F)
job.salary_min,None
job.profile_type,standard
job.contract_duration_max,None
job.status,published


In [65]:
# petite fonction piquée sur le web pour enlever les baslises html
def remove_html_tags(text):
    """Remove html tags from a string"""
    import re
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [76]:
df_job_temp[["job.organization.description","job.profile"]] = df_job_temp[["job.organization.description","job.profile"]].map(remove_html_tags).copy()

In [77]:
df_job_temp.T

,0
job.remote,punctual
job.language,fr
job.published_at,2024-06-10T09:47:07Z
job.experience_level,None
job.reference,DC_5KG03ZX
job.name,Data Manager (H/F)
job.salary_min,None
job.profile_type,standard
job.contract_duration_max,None
job.status,published


Je fais deux fonctions pour décompacter job tools et job skills sans explode mon dataframe

In [103]:
tools = []                                          # je crée un liste pour stocker mes outils
for dicos in (df_job_temp["job.tools"][0]):         # je vais chercher chaque dico contenu dans le premier (et seul) item de ma liste de cette colonne
        for key, tool in dicos.items():             # dans le duo key/value de mes items de chaque dico
            if key == "name":                       # si la clé est = à name
                tools.append(dicos["name"])         # alors je stocke cet outil dans une liste temporaire
print(", ".join(tools))                             # que je convertis en string

Nosql, Sql


In [113]:
# fonction pour décompacter les tools
def jtools_extractor(col):
    tools = []                                      # je crée un liste pour stocker mes outils
    for dicos in (df_job_temp["job.tools"][0]):     # je vais chercher chaque dico contenu dans le premier (et seul) item de ma liste de cette colonne
        for key, tool in dicos.items():             # dans le duo key/value de mes items de chaque dico
            if key == "name":                       # si la clé est = à name
                tools.append(dicos["name"])         # alors je stocke cet outil dans une liste
    # return ", ".join(tools)                       # attention à bien remonter return en dehors de la deuxième boucle for
    return(tools)

In [140]:
df_job_temp["job.tools_clean"] = df_job_temp["job.tools"].apply(jtools_extractor)
df_job_temp

,job.remote,job.language,job.published_at,job.experience_level,job.reference,job.name,job.salary_min,job.profile_type,job.contract_duration_max,job.status,...,job.company_description,job.profile,job.tools,job.skills,job.recruitment_process,job.apply_url,job.wttj_reference,job.contract_duration_min,job.skills_clean,job.tools_clean
0,punctual,fr,2024-06-10T09:47:07Z,None,DC_5KG03ZX,Data Manager (H/F),None,standard,None,published,...,None,"De formation BAC+5, tu as déjà une première ex...","[{'name': 'Nosql', 'reference': '463c82d3-9c2d...",[{'name': {'cs': 'Umělá inteligence a strojové...,None,https://www.davidson.fr/nos-offres/data-manage...,364981ca-b4b0-49f2-9cdc-9c10975b81f4,None,"[Nosql, Sql]","[Nosql, Sql]"


In [137]:
skills = []                                         # je crée un liste pour stocker mes skills
for dictos in (df_job_temp["job.skills"][0]):       # je vais chercher chaque dico contenu dans le premier item de ma liste de cette colonne
    for key, value in dictos.items():               # dans le duo key/value de mes items de chaque dico
        if key == "name":                           # je vais chercher l'item name
           for sec_key, skill in value.items():     # cet item étant un dict. je cherche la paire k/v
               if sec_key == "fr":                  # si la clé est = à name
                     skills.append(skill)           # alors je stocke cette skill dans une liste
print(skills)

['IA et machine-learning', 'Gestion des données', 'Développement de relations', 'Analyse des performances']


In [139]:
# fonction pour décompacter les skills
def jskill_extractor(col):
    skills = []                                         # je crée un liste pour stocker mes skills
    for dictos in (df_job_temp["job.skills"][0]):       # je vais chercher chaque dico contenu dans le premier item de ma liste de cette colonne
        for key, value in dictos.items():               # dans le duo key/value de mes items de chaque dico
            if key == "name":                           # je vais chercher l'item name
               for sec_key, skill in value.items():     # cet item étant un dict. je cherche la paire k/v
                   if sec_key == "fr":                  # si la clé est = à name
                         skills.append(skill)           # alors je stocke cette skill dans une liste
    return(skills)

In [141]:
df_job_temp["job.skills_clean"] = df_job_temp["job.skills"].apply(jskill_extractor)
df_job_temp

,job.remote,job.language,job.published_at,job.experience_level,job.reference,job.name,job.salary_min,job.profile_type,job.contract_duration_max,job.status,...,job.company_description,job.profile,job.tools,job.skills,job.recruitment_process,job.apply_url,job.wttj_reference,job.contract_duration_min,job.skills_clean,job.tools_clean
0,punctual,fr,2024-06-10T09:47:07Z,None,DC_5KG03ZX,Data Manager (H/F),None,standard,None,published,...,None,"De formation BAC+5, tu as déjà une première ex...","[{'name': 'Nosql', 'reference': '463c82d3-9c2d...",[{'name': {'cs': 'Umělá inteligence a strojové...,None,https://www.davidson.fr/nos-offres/data-manage...,364981ca-b4b0-49f2-9cdc-9c10975b81f4,None,"[IA et machine-learning, Gestion des données, ...","[Nosql, Sql]"


In [155]:
# pour la candidature, je ne garde que les mandatory
applications = []                                           # je crée un liste pour stocker mes éléments mandatory
for dicts in (df_job_temp["job.application_fields"][0]):    # je vais chercher chaque dico contenu dans le premier (et seul) item de ma liste de cette colonne
        for key, value in dicts.items():                    # dans le duo key/value de mes items de chaque dico
            if value == "mandatory":                        # si une value est = à mandatory
                applications.append(dicts["name"])          # alors je stocke chaque clé correspondante à name
print(applications)                         

['resume']


In [157]:
# fonction pour récupérer ce qui est obligatoire pour la candidature

def appli_mandatory_extractor(col):
    applications = []                                           # je crée un liste pour stocker mes éléments mandatory
    for dicts in (df_job_temp["job.application_fields"][0]):    # je vais chercher chaque dico contenu dans le premier (et seul) item de ma liste de cette colonne
            for key, value in dicts.items():                    # dans le duo key/value de mes items de chaque dico
                if value == "mandatory":                        # si une value est = à mandatory
                    applications.append(dicts["name"])          # alors je stocke chaque clé correspondante à name
    return(applications)        

In [158]:
df_job_temp["job.application_mandatory"] = df_job_temp["job.application_fields"].apply(appli_mandatory_extractor)
df_job_temp

,job.remote,job.language,job.published_at,job.experience_level,job.reference,job.name,job.salary_min,job.profile_type,job.contract_duration_max,job.status,...,job.profile,job.tools,job.skills,job.recruitment_process,job.apply_url,job.wttj_reference,job.contract_duration_min,job.skills_clean,job.tools_clean,job.application_mandatory
0,punctual,fr,2024-06-10T09:47:07Z,None,DC_5KG03ZX,Data Manager (H/F),None,standard,None,published,...,"De formation BAC+5, tu as déjà une première ex...","[{'name': 'Nosql', 'reference': '463c82d3-9c2d...",[{'name': {'cs': 'Umělá inteligence a strojové...,None,https://www.davidson.fr/nos-offres/data-manage...,364981ca-b4b0-49f2-9cdc-9c10975b81f4,None,"[IA et machine-learning, Gestion des données, ...","[Nosql, Sql]",[resume]


In [159]:
df_jobs_list = df_job_temp.drop(columns=["job.skills", "job.tools", "job.application_fields" ])
df_jobs_list

,job.remote,job.language,job.published_at,job.experience_level,job.reference,job.name,job.salary_min,job.profile_type,job.contract_duration_max,job.status,...,job.start_date,job.company_description,job.profile,job.recruitment_process,job.apply_url,job.wttj_reference,job.contract_duration_min,job.skills_clean,job.tools_clean,job.application_mandatory
0,punctual,fr,2024-06-10T09:47:07Z,None,DC_5KG03ZX,Data Manager (H/F),None,standard,None,published,...,None,None,"De formation BAC+5, tu as déjà une première ex...",None,https://www.davidson.fr/nos-offres/data-manage...,364981ca-b4b0-49f2-9cdc-9c10975b81f4,None,"[IA et machine-learning, Gestion des données, ...","[Nosql, Sql]",[resume]


## # 3. Après le test pour récupérer les données, je lance pour de bon cette 2è étape

Les fonctions

In [243]:
############## Mes fonctions ####################################

### enlever les baslises html
def remove_html_tags(text_col):
    if text_col is None:
        return None
    else:
        clean = re.compile('<.*?>')
        return re.sub(clean,"", text_col)

### décompacter les tools
def jtools_extractor(col):
    tools = []                                      # je crée un liste pour stocker mes outils
    for dicos in (df_jobs_temp["job.tools"][0]):     # je vais chercher chaque dico contenu dans le premier (et seul) item de ma liste de cette colonne
        for key, tool in dicos.items():             # dans le duo key/value de mes items de chaque dico
            if key == "name":                       # si la clé est = à name
                tools.append(dicos["name"])         # alors je stocke cet outil dans une liste
    # return ", ".join(tools)                       # attention à bien remonter return en dehors de la deuxième boucle for
    return(tools)

### décompacter les skills
def jskills_extractor(col):
    skills = []                                         # je crée un liste pour stocker mes skills
    for dictos in (df_jobs_temp["job.skills"][0]):       # je vais chercher chaque dico contenu dans le premier item de ma liste de cette colonne
        for key, value in dictos.items():               # dans le duo key/value de mes items de chaque dico
            if key == "name":                           # je vais chercher l'item name
               for sec_key, skill in value.items():     # cet item étant un dict. je cherche la paire k/v
                   if sec_key == "fr":                  # si la clé est = à name
                         skills.append(skill)           # alors je stocke cette skill dans une liste
    return(skills)

### récupérer ce qui est obligatoire pour la candidature

def appli_mandatory_extractor(col):
    applications = []                                           # je crée un liste pour stocker mes éléments mandatory
    for dicts in (df_jobs_temp["job.application_fields"][0]):    # je vais chercher chaque dico contenu dans le premier (et seul) item de ma liste de cette colonne
            for key, value in dicts.items():                    # dans le duo key/value de mes items de chaque dico
                if value == "mandatory":                        # si une value est = à mandatory
                    applications.append(dicts["name"])          # alors je stocke chaque clé correspondante à name
    return(applications)   

Récupération totale des données

In [165]:
## 1. formatage des urls en relative pour le requêtage ##

relative_urls = [sub.replace("https://www.welcometothejungle.com/fr/companies", "") for sub in jobs_list]
print(relative_urls)
print(len(relative_urls))                               # je vérifie que je n'ai rien perdu

## 2. formatage des urls relatives en url de requêtage ##
api_urls = []
for url in relative_urls:
    api_url = f"https://api.welcometothejungle.com/api/v1/organizations{url}"
    api_urls.append(api_url)
print(api_urls)
print(len(api_urls))                       # je vérifie que je n'ai rien perdu

['/davidson/jobs/data-manager-h-f_DC_5KG03ZX?q=0c297f903b1befdac5f0bebf41aa2676&o=364981ca-b4b0-49f2-9cdc-9c10975b81f4', '/groupe-mentor/jobs/data-analyst-h-f_dommartemont_GM_rXWZkxG?q=0c297f903b1befdac5f0bebf41aa2676&o=110564e2-3db2-42c9-83e7-5510c4c3d8e4', '/groupe-mentor/jobs/data-analyst-h-f_dommartemont_GM_kLL8kRZ?q=0c297f903b1befdac5f0bebf41aa2676&o=e8b049a2-9682-4604-a09a-401f061038f0', '/papernest/jobs/data-analyst-stage-paris_paris?q=0c297f903b1befdac5f0bebf41aa2676&o=78f97400-3f69-4fb5-be5f-319c6aae92ea', '/hublo/jobs/senior-data-analyst-f-h-n_paris_HUBLO_bR85090?q=0c297f903b1befdac5f0bebf41aa2676&o=0b5d40e8-64f9-4643-969b-752d0fd2582c', '/adone/jobs/consultant-senior-data-bi-h-f_paris_AC_bNz65g0?q=0c297f903b1befdac5f0bebf41aa2676&o=823e76d1-9e76-4b00-9bfc-c499f1c7c0ee', '/hsbc/jobs/data-et-analytics-h-f_courbevoie?q=0c297f903b1befdac5f0bebf41aa2676&o=eff1ce84-17be-4859-9741-211a40dea708', '/blablacar/jobs/confirmed-data-analyst-blablacar-daily_paris?q=0c297f903b1befdac5f0beb

In [171]:
## 3.  je requête chaque URL et je stocke dans un dataframe ##

# création d'une liste vide pour stocker les données JSON
data_json = []

for job_url in api_urls:
    link = job_url
    r = requests.get(link)
    data_json.append(json.loads(r.text))                # comme j'itère sur chaque url je dois stocker ce résultat dans une liste
    # print(json.dumps(data_json, indent=2))

# j'en fais un dataframe
df_jraw = pd.json_normalize(data_json)
df_jraw

,job.updated_at,job.remote,job.urls,job.contract_duration_max,job.language,job.contract_type,job.skills,job.slug,job.featured_page.type,job.featured_page.slug,...,job.organization.equality_indexes.gap_in_annual_raises,job.organization.equality_indexes.gap_in_annual_raises_excluding_promotions,job.organization.equality_indexes.gap_in_promotions,job.organization.equality_indexes.gender_pay_gap,job.organization.equality_indexes.maternity_leave_return_raise,job.organization.equality_indexes.published,job.organization.equality_indexes.workforce_range,job.organization.equality_indexes.year,job.featured_page,job.organization.gdpr_setting
0,2024-06-10T09:47:07Z,punctual,"[{'kind': 'canonical', 'href': 'https://www.we...",NaN,fr,full_time,[{'name': {'cs': 'Umělá inteligence a strojové...,data-manager-h-f_DC_5KG03ZX,custom,/culture-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2024-06-10T09:02:59Z,no,"[{'kind': 'canonical', 'href': 'https://www.we...",NaN,fr,full_time,[],data-analyst-h-f_dommartemont_GM_rXWZkxG,culture,/culture,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2024-06-10T11:05:21Z,no,"[{'kind': 'canonical', 'href': 'https://www.we...",NaN,fr,full_time,[],data-analyst-h-f_dommartemont_GM_kLL8kRZ,culture,/culture,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2024-06-11T01:23:39Z,partial,"[{'kind': 'canonical', 'href': 'https://www.we...",NaN,fr,full_time,[],data-analyst-stage-paris_paris,custom,/culture-1,...,35.0,NaN,NaN,8.0,0.0,False,50_250,2023.0,NaN,NaN
4,2024-06-10T07:49:24Z,punctual,"[{'kind': 'canonical', 'href': 'https://www.we...",NaN,fr,full_time,[],senior-data-analyst-f-h-n_paris_HUBLO_bR85090,culture,/culture-2,...,35.0,NaN,NaN,33.0,NaN,False,50_250,2023.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
378,2024-06-07T18:38:53Z,punctual,"[{'kind': 'canonical', 'href': 'https://www.we...",NaN,fr,full_time,[],controleur-de-gestion-h-f_nantes,custom,/culture-1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
379,2024-06-10T23:27:28Z,no,"[{'kind': 'canonical', 'href': 'https://www.we...",NaN,en,full_time,[],solution-specialist-database_paris,plus,/lesplus-1,...,35.0,NaN,NaN,37.0,NaN,False,50_250,2023.0,NaN,NaN
380,2024-06-07T18:44:01Z,partial,"[{'kind': 'canonical', 'href': 'https://www.we...",NaN,fr,full_time,[],expert-dataviz-f-h_lyon-2eme-arrondissement_OR...,plus,/les-plus,...,NaN,20.0,15.0,39.0,15.0,False,1000+,2023.0,NaN,NaN
381,2024-06-11T01:19:26Z,partial,"[{'kind': 'canonical', 'href': 'https://www.we...",NaN,fr,full_time,[],consultant-dataviz-qliksense-f-h_toulouse,plus,/lesplus-1,...,NaN,20.0,15.0,40.0,15.0,False,1000+,2023.0,NaN,NaN


In [236]:
print(df_jraw.columns.values)

['job.updated_at' 'job.remote' 'job.urls' 'job.contract_duration_max'
 'job.language' 'job.contract_type' 'job.skills' 'job.slug'
 'job.featured_page.type' 'job.featured_page.slug' 'job.wttj_reference'
 'job.contract_duration_min' 'job.salary_currency' 'job.salary_min'
 'job.offices' 'job.application_fields' 'job.salary_period'
 'job.recruitment_process' 'job.cta_content.links'
 'job.cta_content.contents' 'job.name' 'job.apply_url' 'job.published_at'
 'job.office.address' 'job.office.local_address' 'job.office.city'
 'job.office.latitude' 'job.office.longitude' 'job.office.district'
 'job.office.country_code' 'job.office.zip_code' 'job.office.local_city'
 'job.office.local_district' 'job.organization.default_language'
 'job.organization.gdpr_setting.application_message'
 'job.organization.gdpr_setting.consent_duration'
 'job.organization.gdpr_setting.privacy_policy_url'
 'job.organization.average_age'
 'job.organization.website_organization.slug'
 'job.organization.website_organization

In [237]:
## 4. je nettoie et je formate ## 

# 4.1 je drop toutes les colonnes dont je n'ai pas besoin
df_jobs_temp = df_jraw.drop(columns=["job.urls", 
                                   "job.slug", 
                                   "job.videos", 
                                   "job.updated_at",
                                   "job.offices", 
                                   "job.is_default",
                                   'job.organization.labels',
                                   'job.organization.logo.url', 
                                   'job.organization.logo.thumb.url',
                                   'job.organization.sectors',
                                   'job.organization.cover_image.small.url',
                                   'job.organization.cover_image.url',
                                   'job.organization.cover_image.medium.url',
                                   'job.organization.cover_image.large.url',
                                   'job.organization.cover_image.social.url',
                                   'job.organization.website_organization.slug',
                                   'job.organization.website_organization.i18n_descriptions.fr',
                                   'job.organization.gdpr_setting.application_message',
                                   'job.organization.gdpr_setting.consent_duration',
                                   'job.organization.gdpr_setting.privacy_policy_url',
                                   'job.organization.headquarter.address',
                                   'job.organization.headquarter.local_address',
                                   'job.organization.headquarter.city',
                                   'job.organization.headquarter.latitude',
                                   'job.organization.headquarter.longitude',
                                   'job.organization.headquarter.district',
                                   'job.organization.headquarter.country_code',
                                   'job.organization.headquarter.zip_code',
                                   'job.organization.headquarter.local_city',
                                   'job.organization.headquarter.local_district',
                                   'job.organization.media_facebook',
                                   'job.organization.media_instagram',
                                   'job.organization.media_linkedin',
                                   'job.organization.media_pinterest',
                                   'job.organization.media_twitter',
                                   'job.organization.media_youtube',
                                   'job.organization.playlist_id',
                                   'job.organization.video_playlist_provider',
                                   'job.organization.automatic_email',
                                   'job.archived_at',
                                   'job.profession.name.cs', 'job.profession.name.en',
                                   'job.profession.name.es', 'job.profession.name.sk',
                                   'job.profession.category.cs',
                                   'job.profession.category.en', 'job.profession.category.es',
                                   'job.profession.category.fr', 'job.profession.category.sk',
                                   'job.profession.category_name.cs',
                                   'job.profession.category_name.en',
                                   'job.profession.category_name.es',
                                   'job.profession.category_name.sk',
                                   'job.profession.sub_category_name.cs',
                                   'job.profession.sub_category_name.en',
                                   'job.profession.sub_category_name.es',
                                   'job.profession.sub_category_name.sk',
                                   'job.profession.sub_category_reference',
                                   'job.questions',
                                   "job.featured_page.type",
                                   'job.featured_page.slug',
                                   'job.cta_content.links',
                                   'job.cta_content.contents',
                                   ]
                                   )



In [240]:
# 4.2 je passe toutes mes fonctions

# html cleaner
### faire un if none pass sinon
df_jobs_temp[["job.organization.description_clean","job.profile_clean"]] = df_jobs_temp[["job.organization.description","job.profile"]].map(remove_html_tags).copy()

# tools extractor
df_jobs_temp["job.tools_clean"] = df_jobs_temp["job.tools"].apply(jtools_extractor)

# skills extractor
df_jobs_temp["job.skills_clean"] = df_jobs_temp["job.skills"].apply(jskills_extractor)

# application mandatories
df_jobs_temp["job.application_mandatory"] = df_jobs_temp["job.application_fields"].apply(appli_mandatory_extractor)


In [245]:
# 4.3 puis je drop les colonnes que j'ai nettoyé
df_jobs = df_jobs_temp.drop(columns=["job.skills", "job.tools", "job.application_fields", "job.profile","job.organization.description_clean"])

In [246]:
df_jobs.sample(5)

,job.remote,job.contract_duration_max,job.language,job.contract_type,job.wttj_reference,job.contract_duration_min,job.salary_currency,job.salary_min,job.salary_period,job.recruitment_process,...,job.organization.equality_indexes.maternity_leave_return_raise,job.organization.equality_indexes.published,job.organization.equality_indexes.workforce_range,job.organization.equality_indexes.year,job.featured_page,job.organization.gdpr_setting,job.profile_clean,job.tools_clean,job.skills_clean,job.application_mandatory
360,no,NaN,fr,full_time,e9fad836-641e-4fba-9d98-a63bc1347ea9,NaN,None,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,Vos compétences et vos atouts : Vous justifiez...,"[Nosql, Sql]","[IA et machine-learning, Gestion des données, ...",[resume]
9,partial,NaN,fr,full_time,c175f679-bcfa-443a-813f-e69242fa561b,NaN,None,NaN,None,None,...,15.0,True,251_999,2023.0,NaN,NaN,Compréhension des enjeux business et réglement...,"[Nosql, Sql]","[IA et machine-learning, Gestion des données, ...",[resume]
189,partial,NaN,fr,full_time,723b1fe9-4067-44d1-bae9-73f68139bb99,NaN,None,NaN,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,None,"[Nosql, Sql]","[IA et machine-learning, Gestion des données, ...",[resume]
120,no,NaN,fr,full_time,97b7f3f3-8079-4137-984d-bd31a3574056,NaN,None,NaN,None,None,...,15.0,False,1000+,2023.0,NaN,NaN,De formation supérieure (école de commerce ou ...,"[Nosql, Sql]","[IA et machine-learning, Gestion des données, ...",[resume]
340,partial,NaN,fr,full_time,234ad906-0036-4db0-abbc-b6c710b2a906,NaN,None,NaN,None,None,...,15.0,True,251_999,2023.0,NaN,NaN,Votre profilD&#39;une formation supérieure en ...,"[Nosql, Sql]","[IA et machine-learning, Gestion des données, ...",[resume]


ici des outils qui m'ont servi mais dont je n'ai plus besoin

In [78]:
extract = list(df_job_temp["job.skills"])
print(extract)

[[{'name': {'cs': 'Umělá inteligence a strojové učení', 'en': 'AI and machine learning', 'es': 'IA y aprendizaje automático', 'fr': 'IA et machine-learning', 'sk': 'Umelá inteligencia a strojové učenie'}, 'reference': '591c17a3-7fb6-4f71-a084-9be16068492c'}, {'name': {'cs': 'Správa dat', 'en': 'Data management', 'es': 'Gestión de datos', 'fr': 'Gestion des données', 'sk': 'Správa údajov'}, 'reference': '6be4470e-23a7-4c01-a6f9-01e55273b41b'}, {'name': {'cs': 'Rozvíjení vztahů', 'en': 'Relationship building', 'es': 'Desarrollar las relaciones', 'fr': 'Développement de relations', 'sk': 'Rozvíjanie vzťahov'}, 'reference': '9f0ddeb6-28f7-4c95-9047-09be059d3a3c'}, {'name': {'cs': 'Analýza výkonu', 'en': 'Performance analysis', 'es': 'Análisis de resultados', 'fr': 'Analyse des performances', 'sk': 'Analýza výkonu'}, 'reference': 'd0144d15-c3f4-49c5-ad86-90af86d9f6f1'}]]


In [79]:
extract = list(df_job_temp["job.tools"])
print(extract)

[[{'name': 'Nosql', 'reference': '463c82d3-9c2d-46ac-8f84-2332c4798f32'}, {'name': 'Sql', 'reference': 'e9255a1d-8071-4a44-a28c-a36d1b925178'}]]


In [142]:
extract = list(df_job_temp["job.application_fields"])
print(extract)

[[{'id': '0e7960ee-b173-4aef-b52d-5c160fe82d22', 'mode': 'optional', 'name': 'cover_letter'}, {'id': '6789a3be-baf8-40d4-b963-fe1235e30bab', 'mode': 'mandatory', 'name': 'resume'}, {'id': 'd464e154-98ad-4d09-87ab-91411aea9fbf', 'mode': 'disabled', 'name': 'portfolio'}, {'id': '7d17fad8-6148-4a6d-ab97-ce41834fe5c6', 'mode': 'optional', 'name': 'picture'}, {'id': '89798d58-34c8-4116-a2f2-bdbe33a86402', 'mode': 'optional', 'name': 'media_website'}, {'id': 'e71d4049-ba42-433c-a899-ff91792f4ffd', 'mode': 'optional', 'name': 'media_linkedin'}, {'id': '5a96b1bc-1841-4aa0-9a59-12e244019fb7', 'mode': 'optional', 'name': 'media_twitter'}, {'id': '7483ddc3-6ec7-47ca-847d-d3f3f2de81a2', 'mode': 'disabled', 'name': 'media_github'}, {'id': '634f797a-dced-4108-b36a-18ba1f82bfe1', 'mode': 'disabled', 'name': 'media_stackoverflow'}, {'id': '3d5c7b20-b39e-4f13-8af3-172f2edcc0aa', 'mode': 'disabled', 'name': 'media_behance'}, {'id': 'c4807109-c7b0-45d9-9112-3dd8401b9d59', 'mode': 'disabled', 'name': 'med

Ci-dessous la poubelle

In [ ]:
## j'abonne la boucle while car il génère des pages vides ad vitam

## fonction selenium qui permet de récupérer l'url courante
# result_url = driver.current_url
# print("The current url is:"+str(result_url))
# # base_url = result_url.replace("1","")

base_url = "https://www.welcometothejungle.com/fr/jobs?query=data%20analyst&refinementList%5Boffices.country_code%5D%5B%5D=FR&refinementList%5Bcontract_type%5D%5B%5D=full_time&refinementList%5Bcontract_type%5D%5B%5D=temporary&page="
page_number = 1
urls_to_scrape = []

while True:
    url = f"{base_url}{page_number}"
    response = requests.get(url)
    if response.status_code == 200:
        print(f"Réponse 200 reçue ! - page :{page_number}")
        page_number += 1
        urls_to_scrape.append(url)
    else:
        print(f"Code de réponse : {response.status_code}. Sortie de la boucle.")
        break

In [ ]:
###code pour récupérer les pages - mais pas valide car href est vide et généré de manière dynamique
# random_delay() 

# je récupère le code HTML de la page avec Selenium
page_source = driver.page_source

#  je passe le code HTML à BeautifulSoup pour l'analyser
soup = BeautifulSoup(page_source, "html.parser")

# Nous filtrons maintenant uniquement sur les acteurs et actrices :
soup_pages = soup.find('nav', {"aria-label" : "Pagination"})


if soup_pages is not None:
# Puis nous cherchons tous les liens contenus dans cet encadré :
    plinks = soup_pages.find_all('a')

# pages = driver.find_elements(By.CLASS_NAME, "sc-dUYLmH")                 # toujours avec Selenium, je cherche les liens ds l'ul sc-cDvQBt eCFBkT
# pages_links = [plink.get_attribute("href") for plink in pages]           # j'extraits le contenu de href pour le mettre dans la liste urls

    print(plinks)

else:
    print("Aucun élément HTML correspondant au sélecteur spécifié n'a été trouvé.")

In [33]:
## je fais une fonction

def urls_extractor(urlss):
    
    if driver.status_code == 200:
        links = driver.find_elements(By.PARTIAL_LINK_TEXT, "Data")          # toujours avec Selenium, je cherche les liens avec data dedans
        urls = [link.get_attribute("href") for link in links]               # j'extraits le contenu de href pour le mettre dans la liste urls
        return(urls)

    else:
        print(f"Erreur lors de la récupération de la page {url}.")

In [54]:
base_url = "https://www.welcometothejungle.com/fr/jobs?refinementList%5Boffices.country_code%5D%5B%5D=FR&query=data%20analyst&page="
page_number = 1
urls_to_scrape = []

while True:
    url = f"{base_url}{page_number}"
    response = requests.get(url)
    if response.status_code == 200:
        print(f"Réponse 200 reçue ! - page :{page_number}")
        page_number += 1
        urls_to_scrape.append(url)
    else:
        print(f"Code de réponse : {response.status_code}. Sortie de la boucle.")
        break


   



Réponse 200 reçue ! - page :1
Réponse 200 reçue ! - page :2
Réponse 200 reçue ! - page :3
Réponse 200 reçue ! - page :4
Réponse 200 reçue ! - page :5
Réponse 200 reçue ! - page :6
Réponse 200 reçue ! - page :7
Réponse 200 reçue ! - page :8
Réponse 200 reçue ! - page :9
Réponse 200 reçue ! - page :10
Réponse 200 reçue ! - page :11
Réponse 200 reçue ! - page :12
Réponse 200 reçue ! - page :13
Réponse 200 reçue ! - page :14
Réponse 200 reçue ! - page :15
Réponse 200 reçue ! - page :16
Réponse 200 reçue ! - page :17
Réponse 200 reçue ! - page :18
Réponse 200 reçue ! - page :19
Réponse 200 reçue ! - page :20
Réponse 200 reçue ! - page :21
Réponse 200 reçue ! - page :22
Réponse 200 reçue ! - page :23


KeyboardInterrupt: 